A practical introduction to Pandas
===================

We have been asked to *analyze an otp dataset*, without much more information. This kind of scenario is more common than you might imagine!

## 1) Quickly examine the files in ~/Data/us_dot/otp. What do they contain, in both technical and functional terms? (Use any tool you want)

In [1]:
path_to_zips = '../data/us_dot/otp/'

In [2]:
path_to_zips = '/home/dsc/Data/us_dot/otp/'

For this example, we are going to begin using the shell from within this notebook, with the ! notation:

A ! sign before a line tells the notebook to send that line straight away to the underlying OS. 

\* Note that we can substitute python variables into the shell command. We do that by surrounding the name of the variable with curly braces ({}). That's what we are going to do with the `path_to_files` variable

### Check what the files contain

In [3]:
!unzip -l {path_to_zips}On_Time_On_Time_Performance_2015_1.zip

Archive:  /home/dsc/Data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
211633432  04-16-2015 20:02   On_Time_On_Time_Performance_2015_1.csv
    12054  04-16-2015 20:02   readme.html
---------                     -------
211645486                     2 files


Oh, it has a readme! Always good to read it.

### Unzip the readme to the current directory

In [4]:
!unzip -o {path_to_zips}On_Time_On_Time_Performance_2015_1.zip readme.html -d .

Archive:  /home/dsc/Data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip
  inflating: ./readme.html           


The readme file is html. Luckily, we are working in an html environment. 

### Display the contents of `readme.html` within the notebook
(Hint: check out IPython.display.IFrame)

In [5]:
from IPython.display import IFrame

In [6]:
IFrame('readme.html', width = 700, height = 500)

That's some very good documentation!

### Summary: 

The files within the zip are " quoted csv's. They contain information on timeliness of departures in the US, at the departure level.

In [7]:
import pandas as pd

Pretty unreadable, so we go for a tool designed specifically for tabular data: **pandas**

### Load some of the data (one of the files) into memory as a pandas dataframe. What functions do you need to use?

Pro tip: there is no need to decompress the whole file. Check out zipfile.ZipFile

First, open a connection to one of the files

In [8]:
import zipfile

In [9]:
zip_file=zipfile.ZipFile(path_to_zips + 'On_Time_On_Time_Performance_2015_1.zip')

In [10]:
csv_file = zip_file.open('On_Time_On_Time_Performance_2015_1.csv')

Now we're ready to load the file into memory as a pandas dataframe. Remember to close the connections to the files!

In [11]:
df = pd.read_csv(csv_file)

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
zip_file.close()
csv_file.close()

Let's start examining the data: show the beginning of the file. How many records does it contain?

In [13]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2015,1,1,1,4,2015-01-01,AA,19805,AA,N787AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,2,5,2015-01-02,AA,19805,AA,N795AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,3,6,2015-01-03,AA,19805,AA,N788AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,7,2015-01-04,AA,19805,AA,N791AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,5,1,2015-01-05,AA,19805,AA,N783AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
len(df)

469968

## Trimming the data

The table is quite wide, and it seems that there are many columns without much data. Which, exactly, are those? (let's consider empty a column that doesn't contain at least 1000 records, arbitrarily)

In [15]:
df[df.columns[df.count()<1000]].count()

DivReachedDest          973
DivActualElapsedTime    684
DivArrDelay             684
DivDistance             971
Div1WheelsOff           704
Div1TailNum             704
Div2Airport              27
Div2AirportID            27
Div2AirportSeqID         27
Div2WheelsOn             27
Div2TotalGTime           27
Div2LongestGTime         27
Div2WheelsOff             9
Div2TailNum               9
Div3Airport               2
Div3AirportID             2
Div3AirportSeqID          2
Div3WheelsOn              2
Div3TotalGTime            2
Div3LongestGTime          2
Div3WheelsOff             0
Div3TailNum               0
Div4Airport               0
Div4AirportID             0
Div4AirportSeqID          0
Div4WheelsOn              0
Div4TotalGTime            0
Div4LongestGTime          0
Div4WheelsOff             0
Div4TailNum               0
Div5Airport               0
Div5AirportID             0
Div5AirportSeqID          0
Div5WheelsOn              0
Div5TotalGTime            0
Div5LongestGTime    

In [16]:
count = df.count()

In [17]:
escasos = count[count<1000]

It seems that the "diverted" fields, after the first, are often empty. No big surprise, since not that many flights must be diverted more than once in a month. Let's drop those columns, since we are not that interested in those, at least for now.

In [18]:
df.drop(escasos.index,axis = 1, inplace = True)

In [19]:
len(df.columns)

71

Now that we have eliminated some inconvenient data columns, let's have a look at the rest:

Let's loook at how the location data is encoded.

### Select the columns that have 'Origin' in their name

Hint: we are going to use the str attribute of Series and Indexes

In [20]:
col_containin_origin = df.columns[df.columns.str.contains('Origin')]

Let's now have a look at these. We have been using the .head() method, but that always returns the same values, so it's not that good for getting a feel for the data. Let's start using a better one, that will give us a sample of the data: df.sample()

In [21]:
df[col_containin_origin].sample(5)

,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac
455754,12478,1247802,31703,JFK,"New York, NY",NY,36,New York,22
190824,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91
107403,13198,1319801,33198,MCI,"Kansas City, MO",MO,29,Missouri,64
392959,10792,1079204,30792,BUF,"Buffalo, NY",NY,36,New York,22
261127,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43


So much redundant information!!!!! Let's drop a few of those fields. Remember, 90% of the time spent in Data Science is data cleaning... 

### Do the same for the destination columns

In [22]:
col_containin_dest = df.columns[df.columns.str.contains('Dest')]

In [23]:
df[col_containin_dest].sample(5)

,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac
183224,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33
122600,10713,1071302,30713,BOI,"Boise, ID",ID,16,Idaho,83
178864,13232,1323202,30977,MDW,"Chicago, IL",IL,17,Illinois,41
123531,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36
228888,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87


### Now, discard all these columns with redundant information. 

The set of columns we want to discard is 

`df.columns[col_contains_dest] + df.columns[col_contains_origin] - columns_of_interest`

We'll also use the opportunity to drop Year, Month, Day and Quarter columns since that is information that is already contained in FlightDate. We'll keep the DayOfWeek, though.

In [24]:
columns_of_interest = pd.Index(['Origin','OriginCityName','OriginStateName','Dest','DestCityName','DestStateName'])

In [25]:
cols_to_drop = pd.Index(['Year','Month','DayofMonth','Quarter'])

In [26]:
cols_to_drop = cols_to_drop | col_containin_origin | col_containin_dest

In [27]:
cols_to_drop = cols_to_drop.difference(columns_of_interest)

In [28]:
df.drop(cols_to_drop, axis = 1, inplace=True)

In [29]:
df.sample(5)

,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,Origin,OriginCityName,OriginStateName,...,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime
29515,4,2015-01-08,AA,19805,AA,N485AA,1600,DFW,"Dallas/Fort Worth, TX",Texas,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
411990,3,2015-01-07,EV,20366,EV,N916EV,5404,BTR,"Baton Rouge, LA",Louisiana,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
247758,2,2015-01-13,OO,20304,OO,N743SK,6198,DEN,"Denver, CO",Colorado,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
51492,1,2015-01-26,AS,19930,AS,N763AS,64,JNU,"Juneau, AK",Alaska,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
194844,3,2015-01-21,WN,19393,WN,N293WN,1811,OAK,"Oakland, CA",California,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


OK, so we have cleaned up most of the date / location information,  and some of the extra information that was not really useful, in the process reducing the dataframe from 110 columns to 60. Let's look at the really interesting info: delays.

### Select all columns that contain 'Delay' in their column name

In [30]:
delay_column = df.columns[df.columns.str.contains('Delay')]
delay_column

Index([u'DepDelay', u'DepDelayMinutes', u'DepartureDelayGroups', u'ArrDelay',
       u'ArrDelayMinutes', u'ArrivalDelayGroups', u'CarrierDelay',
       u'WeatherDelay', u'NASDelay', u'SecurityDelay', u'LateAircraftDelay'],
      dtype='object')

In [31]:
df[delay_column].sample(5)

,DepDelay,DepDelayMinutes,DepartureDelayGroups,ArrDelay,ArrDelayMinutes,ArrivalDelayGroups,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
162416,-5,0,-1,-20,0,-2,NaN,NaN,NaN,NaN,NaN
339221,0,0,0,11,11,0,NaN,NaN,NaN,NaN,NaN
442233,17,17,1,46,46,3,0,0,29,0,17
283677,-7,0,-1,-1,0,-1,NaN,NaN,NaN,NaN,NaN
57375,83,83,5,83,83,5,83,0,0,0,0


OK, we are going to center our attention in the DepDelay column, so we won't need the others. 

### Drop those too. 

We already know the drill, don't we?

In [32]:
df.drop(delay_column.difference(['DepDelay']), axis=1, inplace=True)

We are not that interested in the 'Diverted' columns either, so let's drop those too

In [33]:
df.columns[df.columns.str.contains('Div')]

Index([u'Diverted', u'DivAirportLandings', u'Div1Airport', u'Div1AirportID',
       u'Div1AirportSeqID', u'Div1WheelsOn', u'Div1TotalGTime',
       u'Div1LongestGTime'],
      dtype='object')

In [34]:
df.drop(df.columns[df.columns.str.startswith('Div')], inplace=True, axis=1)

All of this is getting a little repetitive and boring, so let's just specify the columns we want:

In [35]:
df = df[['FlightDate', 'DayOfWeek', 'Carrier', 'TailNum', 'FlightNum', 'Origin', 
         'OriginCityName', 'OriginStateName', 'Dest', 'DestCityName', 'DestStateName',
         'DepTime', 'DepDelay', 'AirTime', 'Distance']]
df.sample(5)

,FlightDate,DayOfWeek,Carrier,TailNum,FlightNum,Origin,OriginCityName,OriginStateName,Dest,DestCityName,DestStateName,DepTime,DepDelay,AirTime,Distance
460643,2015-01-16,5,MQ,N662MQ,3459,PNS,"Pensacola, FL",Florida,MIA,"Miami, FL",Florida,1806,11,82,530
446963,2015-01-04,7,MQ,N665MQ,3147,CMH,"Columbus, OH",Ohio,MIA,"Miami, FL",Florida,2105,190,150,990
365386,2015-01-26,1,DL,N950DL,2459,JFK,"New York, NY",New York,MCO,"Orlando, FL",Florida,1653,52,135,944
151823,2015-01-07,3,WN,N361SW,489,MDW,"Chicago, IL",Illinois,BNA,"Nashville, TN",Tennessee,1342,22,53,395
245597,2015-01-04,7,EV,N933EV,5107,ATL,"Atlanta, GA",Georgia,GSP,"Greer, SC",South Carolina,1105,40,37,153


## Formatting columns and parsing dates and times

Hurray! we have almost cleaned our dataset. Soon we will begin to do some actual work with it. 

Pandas interpreted the Deptime column as ints and the FlighDate column as strings. We want to combine them and parse them into a DateTime column, so that we can use them properly as datetimes.

First, 

### Define a function that will parse our int hours into a reasonable format ("HH:MM")

In [36]:
time_to_proper_format = lambda x: '%02d:%02d' % ((x / 100) % 24 , x% 100)

Now, we can use that function to build datetime strings that we will then pass to pd.to_datetime, with a format we will specify. Let's do that

Hint: Check out pd.to_datetime

### Overwrite the 'DepTime' column with its version in the proper format

Hint: Before overwriting your column in the dataframe, make sure that everything works by assigning the modified column to a Series variable.

In [37]:
df['DepTime'] = df['DepTime'].fillna(0.0).map(time_to_proper_format)
df.sample(5)

,FlightDate,DayOfWeek,Carrier,TailNum,FlightNum,Origin,OriginCityName,OriginStateName,Dest,DestCityName,DestStateName,DepTime,DepDelay,AirTime,Distance
343823,2015-01-13,2,OO,N792SK,5620,DFW,"Dallas/Fort Worth, TX",Texas,LAX,"Los Angeles, CA",California,16:17,-2,201,1235
104576,2015-01-15,4,DL,N329NB,1708,MSP,"Minneapolis, MN",Minnesota,DEN,"Denver, CO",Colorado,09:05,-5,91,680
90982,2015-01-08,4,DL,N981AT,1733,ATL,"Atlanta, GA",Georgia,ECP,"Panama City, FL",Florida,11:30,25,40,240
86658,2015-01-07,3,DL,N937DN,104,ATL,"Atlanta, GA",Georgia,BOS,"Boston, MA",Massachusetts,07:25,0,125,946
161252,2015-01-10,6,WN,N658SW,3782,BWI,"Baltimore, MD",Maryland,DAY,"Dayton, OH",Ohio,12:31,1,76,406


### Now, create a DepDateTime with the proper type using `pd.to_datetime`

In [38]:
df['DepDateTime'] = departure_datetimes = pd.to_datetime(df['FlightDate'] + ' ' + df['DepTime'], format='%Y-%m-%d %H:%M')
df.drop(['FlightDate','DepTime'], axis=1, inplace=True)

In [39]:
df.sample(5)

,DayOfWeek,Carrier,TailNum,FlightNum,Origin,OriginCityName,OriginStateName,Dest,DestCityName,DestStateName,DepDelay,AirTime,Distance,DepDateTime
324974,7,UA,N496UA,442,IAH,"Houston, TX",Texas,LGA,"New York, NY",New York,39,167,1416,2015-01-04 13:42:00
212027,7,WN,N468WN,4208,SMF,"Sacramento, CA",California,PDX,"Portland, OR",Oregon,-3,74,479,2015-01-25 14:17:00
306715,1,UA,N27205,1493,EWR,"Newark, NJ",New Jersey,LAX,"Los Angeles, CA",California,-3,356,2454,2015-01-19 14:27:00
2324,2,AA,N3DPAA,63,MIA,"Miami, FL",Florida,PHX,"Phoenix, AZ",Arizona,6,260,1972,2015-01-06 20:01:00
233495,6,WN,N275WN,4438,MCO,"Orlando, FL",Florida,ALB,"Albany, NY",New York,-5,153,1073,2015-01-31 11:20:00


### Inspect the types, see if everything is in order

In [40]:
df.dtypes

DayOfWeek                   int64
Carrier                    object
TailNum                    object
FlightNum                   int64
Origin                     object
OriginCityName             object
OriginStateName            object
Dest                       object
DestCityName               object
DestStateName              object
DepDelay                  float64
AirTime                   float64
Distance                  float64
DepDateTime        datetime64[ns]
dtype: object

### Transform the FlightNum column into a column of strings

In [41]:
df['FlightNum'] = df['FlightNum'].map(str)

In [42]:
df.dtypes

DayOfWeek                   int64
Carrier                    object
TailNum                    object
FlightNum                  object
Origin                     object
OriginCityName             object
OriginStateName            object
Dest                       object
DestCityName               object
DestStateName              object
DepDelay                  float64
AirTime                   float64
Distance                  float64
DepDateTime        datetime64[ns]
dtype: object

## Examining the data

Finally clean! Let's start to do some preliminary work on the data.

### Find the biggest delays. 

How would you find the 5 maximum delays?

In [43]:
delays = df['DepDelay'].copy()
delays.head()

0    -5
1   -10
2    -7
3    -7
4    -7
Name: DepDelay, dtype: float64

In [44]:
delays.sort(ascending=False)
topdelays = delays.head()

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting
  if __name__ == '__main__':


In [45]:
df.ix[topdelays.index]

,DayOfWeek,Carrier,TailNum,FlightNum,Origin,OriginCityName,OriginStateName,Dest,DestCityName,DestStateName,DepDelay,AirTime,Distance,DepDateTime
21523,5,AA,N598AA,1322,BHM,"Birmingham, AL",Alabama,DFW,"Dallas/Fort Worth, TX",Texas,1988,99,597,2015-01-23 16:08:00
11524,7,AA,N5DGAA,224,LAS,"Las Vegas, NV",Nevada,LAX,"Los Angeles, CA",California,1604,42,236,2015-01-18 14:14:00
18198,2,AA,N586AA,1242,FAT,"Fresno, CA",California,DFW,"Dallas/Fort Worth, TX",Texas,1551,155,1313,2015-01-27 08:50:00
108152,3,AA,N4XNAA,2385,JAX,"Jacksonville, FL",Florida,DFW,"Dallas/Fort Worth, TX",Texas,1457,144,918,2015-01-21 12:40:00
26001,7,AA,N469AA,1595,AUS,"Austin, TX",Texas,DFW,"Dallas/Fort Worth, TX",Texas,1450,37,190,2015-01-11 07:00:00


### What was the average delay for this month? Standard deviation and typical value?

In [46]:
df['DepDelay'].describe()

count    458311.000000
mean          9.759471
std          36.636491
min         -48.000000
25%          -5.000000
50%          -2.000000
75%           9.000000
max        1988.000000
Name: DepDelay, dtype: float64

Also: a quick look at the correlation between the numerical variables is extremely easy with pandas:

In [47]:
df.corr()

,DayOfWeek,DepDelay,AirTime,Distance
DayOfWeek,1.000000,0.017254,0.027161,0.021530
DepDelay,0.017254,1.000000,0.013282,0.013871
AirTime,0.027161,0.013282,1.000000,0.981376
Distance,0.021530,0.013871,0.981376,1.000000


### A little taste of groupby

Very often, we will want to split our data according to a variable, then compute some statistics on the different groups. We will see this in depth next week, but I want to give you a little taste.

What is the plane that has the highest average delay? We'll first group by tail number (the *license plate* of a plane) and then calculate the relevant statistic for each group (group of *departures*)

In [48]:
grouped_by_plane = df.groupby('TailNum')

In [52]:
meandelays = grouped_by_plane['DepDelay'].mean().sort_values(ascending = False)
meandelays.head()

TailNum
N79011    232.0
N228UA    217.5
N707DN    204.0
N651UA    167.0
N776UA    159.0
Name: DepDelay, dtype: float64

Oh, but those numbers smell like these planes had only a few, very delayed, departures! how can we count the number of departures *and* calculate the average delay at the same time?

In [53]:
delaybyplane = grouped_by_plane['DepDelay'].agg(['count', 'mean'])

Awesome! now, let's look at the average delay of the planes with some departures (let's say, at least 15)

In [57]:
delaybyplane[delaybyplane['count']>14].sort_values('mean', ascending = False).head()

,count,mean
TailNum,,
N398AA,15,118.733333
N5FSAA,15,105.800000
N557AA,23,85.434783
N12125,34,67.235294
N189DN,20,65.800000


## Plotting

There are several ways in which we could go about this. 

For example, do the delays have a relationship with the number of departures a plane does?

 Another question: how do the delays stack over the course of the day?

## Self study:

Now, you can try the above with the whole dataset. First, we need to join all the zipped csvs into one dataframe. How would you do that? Do you think you could run into memory problems? How would you avoid that?

Hints for reading large files: 
* selecting columns:
    `pd.read_csv(usecols=[list_of_interesting_column_names])`
* bcolz
* specify column types